In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
UP = -1
DOWN = 1
LEFT = -1j
RIGHT = 1j


class SnakeBody(object):
    def __init__(self, x, y):
        self.pos = x + y *1j
        self.next = None
        
    def __repr__(self):
        return "(%d, %d)" % (self.pos.real, self.pos.imag)
    
    def __contains__(self, pos):
        ptr = self
        while ptr:
            if pos == ptr.pos:
                return True
            ptr = ptr.next
        return False
        

class Snake:
    def __init__(self, org_x, org_y, length=5, space_size=100):
        self.head = None
        self.direction = RIGHT
        self.init_body(org_x, org_y, length)
        self.space_size = space_size
    
    
    def move(self, extend=False):
        new_pos = self.head.pos + self.direction
        new_head = SnakeBody(new_pos.real % self.space_size, new_pos.imag % self.space_size)
        new_head.next = self.head
        self.head = new_head
        if not extend:
            ptr = self.head.next
            prev = self.head
            while ptr.next:
                prev = ptr
                ptr = ptr.next
            prev.next = None


    def init_body(self, org_x, org_y, size):
        self.head = SnakeBody(org_x, org_y)
        ptr = self.head
        for i in range(int(size)):
            if i == size - 1:
                self.tail = ptr
            ptr.next = SnakeBody(org_x, org_y - i -1)
            ptr = ptr.next
        
        
    def on_key(self, event):
        directions = {'up': UP, 'down': DOWN, 'left': LEFT, 'right': RIGHT}
        if event.key in directions:
            if directions[event.key] != -self.direction:
                self.direction = directions[event.key]

    def __contains__(self, pos):
        return pos in self.head
    

In [6]:
import random
import time


INIT = 0
PLAYING = 1
PAUSE = 2
END = 3


class Game(object):
    def __init__(self, size=100, t_sleep=0.001, t_pause=0.001):
        self.size = size
        self.t_sleep = t_sleep
        self.t_pause = t_pause
        self.init_game()
        self.init_graphic()
        plt.show()

        
    def init_game(self):
        self.state = INIT
        self.score = 0
        self.snake = Snake(self.size/2, self.size/2, self.size/3, self.size)
        self.food = self.place_food()

        
    def init_graphic(self):
        fig = plt.figure()
        fig.set_size_inches(5, 5)
        plt.xlim(self.size, self.size)
        plt.ylim(self.size, self.size)
        plt.axis('off')
        cid = fig.canvas.mpl_connect('key_press_event', self.on_key)
        self.render()
        
        
    def on_key(self, event):
        if self.state == INIT:
            self.state = PLAYING
            self.start()
        elif self.state == PLAYING:
            if event.key == ' ':
                self.state = PAUSE
            else:
                self.snake.on_key(event)
        elif self.state == PAUSE:
            self.state = PLAYING
            self.start()
        elif self.state == END:
            self.init_game()
 

            
    def start(self):
        while plt.get_fignums() and self.state == PLAYING:
            if self.hit():
                self.state = END
                break
            elif self.eat():
                self.score += 1

                self.snake.move(extend=True)
                self.food = self.place_food()
            else:
                self.snake.move()
            self.render()
            time.sleep(self.t_sleep)
            plt.pause(self.t_pause)

            
            
    def place_food(self):
        x = random.randint(0, self.size - 1)
        y = random.randint(0, self.size - 1)
        while (x + y*1j) in self.snake:
            x = random.randint(0, self.size - 1)
            y = random.randint(0, self.size - 1)
        return x + y*1j


    def render(self):
        plt.cla()
        plt.axis('off')
        ptr = self.snake.head
        mat = np.zeros((self.size, self.size))
        while ptr:
            mat[ptr.pos.real, ptr.pos.imag] = 1
            ptr = ptr.next
        mat[self.food.real, self.food.imag] = .5
        plt.imshow(mat, interpolation='nearest', cmap=plt.cm.Greys)
        plt.title("score : {:,}".format(self.score))
        plt.draw()
        
        
    def eat(self):
        return self.snake.head.pos == self.food

    
    def hit(self):
        return self.snake.head.pos in self.snake.head.next
    

In [9]:
g = Game(100, t_sleep=0.0001, t_pause=0.0001)

/home/kutay/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_base.py:2787: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=100, right=100
  'left=%s, right=%s') % (left, right))
/home/kutay/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_base.py:3045: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=100, top=100
  'bottom=%s, top=%s') % (bottom, top))
/home/kutay/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:88: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/kutay/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:90: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/kutay/anaconda3/lib/python3.5/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWar